In [ ]:
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.datasets import LIBRISPEECH

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAMPLE_RATE = 16000
N_MFCC = 13
N_MELS = 80
BLANK_ID = 0

torch.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# =========================
# 2) Vocab & encodage
# =========================
VOCAB = [" ", "'"] + [chr(i) for i in range(97, 123)]  # " ", "'", a..z
char2id = {c: i + 1 for i, c in enumerate(VOCAB)}      # 1.. (0 = blank)
id2char = {i + 1: c for i, c in enumerate(VOCAB)}

def normalize_text(txt: str) -> str:
    txt = txt.lower()
    return "".join(ch for ch in txt if ch in char2id)

def text_to_int(txt: str):
    return [char2id[ch] for ch in normalize_text(txt)]

def int_to_text(ids):
    return "".join(id2char[i] for i in ids if i in id2char)

In [ ]:
# =========================
# 3) MFCC & resample
# =========================
mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=SAMPLE_RATE,
    n_mfcc=N_MFCC,
    melkwargs={"n_fft": 400, "hop_length": 160, "n_mels": 40, "center": False},
)

_resamplers = {}
def maybe_resample(waveform: torch.Tensor, sr: int) -> torch.Tensor:
    if sr == SAMPLE_RATE:
        return waveform
    if sr not in _resamplers:
        _resamplers[sr] = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SAMPLE_RATE)
    return _resamplers[sr](waveform)

In [ ]:
# =========================
# 4) Modèle MLP + CTC
# =========================
class MLPCTC(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        B, T, Fdim = x.shape
        x = x.reshape(B * T, Fdim)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        x = x.view(B, T, -1).transpose(0, 1)
        return x

In [ ]:
# =========================
# 5) Collate function
# =========================
def collate_fn(batch):
    """
    batch: list of tuples (waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id)
    Retour:
      - features: (B, T_max, F)
      - input_lengths: (B,)
      - targets_concat: (sum_S,)
      - target_lengths: (B,)
      - texts: liste des textes normalisés (debug/affichage)
    """
    feats_list = []
    input_lengths = []
    targets = []
    target_lengths = []
    texts = []

    for waveform, sr, transcript, *_ in batch:
        if waveform.dim() == 2:
            waveform = waveform.mean(dim=0)
        else:
            waveform = waveform.squeeze(0)

        # Resample si besoin
        waveform = maybe_resample(waveform, int(sr))

        mfcc = mfcc_transform(waveform)
        if mfcc.dim() == 3:
            mfcc = mfcc.squeeze(0)
        feat = mfcc.transpose(0, 1).contiguous()

        feats_list.append(feat)
        input_lengths.append(feat.shape[0])

        norm_txt = normalize_text(transcript)
        y = torch.tensor(text_to_int(norm_txt), dtype=torch.long)
        targets.append(y)
        target_lengths.append(len(y))
        texts.append(norm_txt)


    F = feats_list[0].shape[1]
    T_max = max(t.shape[0] for t in feats_list)
    padded = torch.zeros(len(feats_list), T_max, F, dtype=feats_list[0].dtype)
    for i, f in enumerate(feats_list):
        padded[i, : f.shape[0]] = f

    features = padded
    input_lengths = torch.tensor(input_lengths, dtype=torch.long)
    targets_concat = torch.cat(targets) if len(targets) > 0 else torch.empty(0, dtype=torch.long)
    target_lengths = torch.tensor(target_lengths, dtype=torch.long)

    return features, input_lengths, targets_concat, target_lengths, texts


In [ ]:
# =========================
# 6) Greedy CTC decode
# =========================
def greedy_ctc_decode(log_probs: torch.Tensor, input_lengths: torch.Tensor):
    """
    log_probs: (T, B, C)
    input_lengths: (B,)
    """
    max_ids = log_probs.argmax(dim=-1)  # (T, B)
    T, B = max_ids.shape
    results = []
    for b in range(B):
        prev = None
        seq = []
        for t in range(int(input_lengths[b].item())):
            p = int(max_ids[t, b].item())
            if p != BLANK_ID and p != prev:
                seq.append(p)
            prev = p
        results.append(int_to_text(seq))
    return results

In [ ]:
# =========================
# 7) Entraînement
# =========================
def train_one_epoch(model, loader, optimizer, criterion, epoch, log_interval=50, grad_clip=5.0):
    model.train()
    running = 0.0
    for step, (features, input_lengths, targets_concat, target_lengths, texts) in enumerate(loader, 1):
        features = features.to(DEVICE)
        input_lengths = input_lengths.to(DEVICE)
        targets_concat = targets_concat.to(DEVICE)
        target_lengths = target_lengths.to(DEVICE)

        logits = model(features)
        log_probs = logits.log_softmax(dim=2)

        loss = criterion(log_probs, targets_concat, input_lengths, target_lengths)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        if grad_clip is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        running += loss.item()
        if step % log_interval == 0:
            avg = running / log_interval
            print(f"[epoch {epoch} step {step}] loss={avg:.4f}")
            running = 0.0

            # Décodage rapide sur le mini-batch courant
            with torch.no_grad():
                preds = greedy_ctc_decode(log_probs.detach(), input_lengths)
            # Affiche deux exemples
            for i in range(min(2, len(preds))):
                tgt = texts[i]
                print(f"  tgt: {tgt}")
                print(f"  prd: {preds[i]}")

# CNN

In [ ]:
# =========================
# 3) Transforms: resample + MelSpectrogram
# =========================
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=400,
    hop_length=160,
    n_mels=N_MELS,
    center=False,
    power=2.0,
)

_resamplers = {}
def maybe_resample(waveform: torch.Tensor, sr: int) -> torch.Tensor:
    """
    Resample vers SAMPLE_RATE si besoin.
    waveform: (time,) ou (1, time) ou (channels, time)
    """
    if sr == SAMPLE_RATE:
        return waveform
    if sr not in _resamplers:
        _resamplers[sr] = torchaudio.transforms.Resample(
            orig_freq=sr, new_freq=SAMPLE_RATE
        )
    return _resamplers[sr](waveform)

In [ ]:
# =========================
# 4) Collate function pour CNN + MelSpec
# =========================
def collate_fn_cnn(batch):
    specs = []
    input_lengths = []
    targets = []
    target_lengths = []
    texts = []

    for waveform, sr, transcript, *_ in batch:
        if waveform.dim() == 2:
            waveform = waveform.mean(dim=0)
        else:
            waveform = waveform.squeeze(0)

        # Resample si besoin
        waveform = maybe_resample(waveform, int(sr))

        mel = mel_transform(waveform)
        if mel.dim() == 3:
            mel = mel.squeeze(0)

        n_mels, T_spec = mel.shape
        specs.append(mel)
        input_lengths.append(T_spec)

        norm_txt = normalize_text(transcript)
        y = torch.tensor(text_to_int(norm_txt), dtype=torch.long)
        targets.append(y)
        target_lengths.append(len(y))
        texts.append(norm_txt)

    B = len(specs)
    n_mels = specs[0].shape[0]
    T_max = max(s.shape[1] for s in specs)

    features = torch.zeros(B, 1, n_mels, T_max, dtype=specs[0].dtype)
    for i, s in enumerate(specs):
        T_i = s.shape[1]
        features[i, 0, :, :T_i] = s

    input_lengths = torch.tensor(input_lengths, dtype=torch.long)
    targets_concat = torch.cat(targets) if len(targets) > 0 else torch.empty(0, dtype=torch.long)
    target_lengths = torch.tensor(target_lengths, dtype=torch.long)

    return features, input_lengths, targets_concat, target_lengths, texts

In [ ]:
# =========================
# 5) Modèle CNN + CTC
# =========================
class ConvCTC(nn.Module):
    """
    Modèle CNN + CTC:
      Entrée:  (B, 1, n_mels, T)
      Sortie:  (T, B, num_classes)
    """
    def __init__(self, n_mels: int, num_classes: int, hidden_dim: int = 256):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=(2, 1)),
        )

        self.n_mels_after = n_mels // 2
        conv_feat_dim = 64 * self.n_mels_after
        self.fc = nn.Linear(conv_feat_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.conv(x)
        B, C, Freq, T = x.shape


        x = x.permute(0, 3, 1, 2).contiguous()
        x = x.view(B, T, C * Freq)

        x = F.relu(self.fc(x))
        x = self.out(x)

        x = x.transpose(0, 1)
        return x

In [ ]:
train_dataset = LIBRISPEECH("./data", url="dev-clean", download=True)

# DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4
    collate_fn=collate_fn_cnn,
    pin_memory=True if DEVICE.type == "cuda" else False,
)

num_classes = 1 + len(VOCAB)      # 0=blank + 1.. vocab
# model = MLPCTC(input_dim=N_MFCC, hidden_dim=256, num_classes=num_classes).to(DEVICE)
model = ConvCTC(n_mels=N_MELS, num_classes=num_classes, hidden_dim=256).to(DEVICE)
criterion = nn.CTCLoss(blank=BLANK_ID, zero_infinity=True).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 50
for ep in range(1, epochs + 1):
    train_one_epoch(model, train_loader, optimizer, criterion, ep, log_interval=200)


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#t

[epoch 1 step 50] loss=4.9176
  tgt: dear good god help us now she prayed
  prd: 
  tgt: mister meekin expressed some alarm but doctor macklewain re assured him
  prd: 
[epoch 1 step 100] loss=3.2175
  tgt: i am a minister of god sir and i forbid you to commit this crime
  prd: 
  tgt: in aristocratic countries the great possess immense privileges upon which their pride rests without seeking to rely upon the lesser advantages which accrue to them
  prd: 
[epoch 1 step 150] loss=3.2551
  tgt: i should have a violent attack of the cramp if i were not to have some sort of exercise
  prd: 
  tgt: i'm here because the matter is of utmost importance and brandd is the one i must see now stand aside
  prd: 
[epoch 1 step 200] loss=3.2677
  tgt: indeed it is not a nest at all only the beginning of one
  prd: 
  tgt: randal he said you know where sydney is
  prd: 
[epoch 1 step 250] loss=3.1723
  tgt: many little wrinkles gathered between his eyes as he contemplated this and his brow moistened
 

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

[epoch 2 step 50] loss=2.9804
  tgt: in the valley of the shadow of death he is with us
  prd: 
  tgt: we saw the united states flag flying from the ramparts and thought that yank would probably be asleep or catching lice or maybe engaged in a game of seven up
  prd: 
[epoch 2 step 100] loss=2.9590
  tgt: who is touching me and untrussing me
  prd: 
  tgt: by which a soul is drawn from its body and across gulfs of echoing space returned the man on the mat
  prd: 
[epoch 2 step 150] loss=2.9406
  tgt: gentlemen he said in a hoarse voice give me your word of honor that this horrible secret shall forever remain buried amongst ourselves the two men drew back
  prd:       
  tgt: general halleck in speaking of this battle said
  prd: 
[epoch 2 step 200] loss=2.9277
  tgt: i saw the lady who erewhile appeared veiled underneath the angelic festival direct her eyes to me across the river
  prd:     
  tgt: ay she answered half bitterly and would that we could live without food and imitate perf

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#t

...

[epoch 48 step 50] loss=1.5745
  tgt: cosette on her side had also unknown to herself become another being poor little thing
  prd: coatancersoa olsom onerilbecoa notr begw loe fhen 
  tgt: gwynplaine ate up their public
  prd: wyn plainat up therpit
[epoch 48 step 100] loss=1.6051
  tgt: yet there are a few private rooms which contain a table surrounded with benches in which a respectable family or a few friends can enjoy themselves in a decent way
  prd: ytherfe favereswe t o tantabosovrn de e banesinwe eesuotfe bofe maro frns eantothesos tete on
  tgt: one could see that he was very much wrapped up in his offspring
  prd: lin ofee ta hews vernma vreptobans vofri
[epoch 48 step 150] loss=1.6278
  tgt: a young fellow who had been hovering in the background at once stepped forward
  prd: a a elo ihabeinrnnhaboetatonstoptfod
  tgt: the federal army was concentrating at nashville there was no rest for the weary
  prd: hefet ernmy was pison redlin notd nase was nherro st fithe war
[epoch 

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

[epoch 49 step 50] loss=1.6298
  tgt: then the hills were piled up on top of it all but here and there some edge of a coal bed was tilted up and appeared above the ground
  prd: ee po no oeo  aoo o e  po e  en e pe
  tgt: at some uncertain period in the depths of night and as it were through the thin veil of a dream she was conscious of a footstep mounting the stairs heavily but not with force and decision
  prd: astamonsrrpertinetat no n as wa o e tanaoinesonhso efete nawnteng i asinao eo ot iforan desi
[epoch 49 step 100] loss=1.5678
  tgt: as they sat down a burst of applause drew alexander's attention to the stage
  prd: as the hattan  aursthe o tlohe  aasxn asan stohecthe 
  tgt: she would take the envelope and know that she had triumphed
  prd: owal tak o eloan nathit she atdti 
[epoch 49 step 150] loss=1.6209
  tgt: but how to get him out was the unsolved problem
  prd: at howta gnot m mot wase nsol prub elo 
  tgt: illustration rusks
  prd: ellustraton resks
[epoch 49 step 200]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#t

[epoch 50 step 50] loss=1.5703
  tgt: and if you have time it would be a great service to translate the analyses of the poems which i omitted
  prd: in f ha tinm i ibe a cretseesto trinslathanoss o epon wih im mat end  
  tgt: irolg looked amazed at the sudden fury of the attack then smiled
  prd: ro tmasdtse se no e atakthn s my
[epoch 50 step 100] loss=1.6164
  tgt: he had never been father lover husband friend
  prd: he knhaerenfot e loferosi ron
  tgt: so that on that score there is little or no danger
  prd: oa taorthe leloe moteei
[epoch 50 step 150] loss=1.5907
  tgt: this was not the fashion of a year ago when they exchanged a friendly word or two but harry knew its cause now nobody could trust anybody else
  prd: theh s note fehioeteur geen matudo frer a wordwbe r natt csnan nowtyorouse in hbhothewis
  tgt: on the dais under the golden dome the king cried out again racked by awful paroxysms
  prd: onm lhe vs oner bhe gaden dhominrid at egimrak o fo proxiss
[epoch 50 step 200] 